# Size a HPP plant based on a simplified hpp model

This **static** notebook illustrates how to solve hybrid power plant sizing optimization in a specific location based on preselected turbine type.

**To execute this setup a server is required as it relies on paralle evaluation of the model.**

Sizing a hybrid power plant consists on designing the following parameters:

### Design Variables

**Wind Plant design:**

- Number of wind turbines in the wind plant [-] (`Nwt`)
- Wind power installation density [MW/km2] (`wind_MW_per_km2`): This parameter controls how closely spaced are the turbines, which in turns affect how much wake losses are present.

**PV Plant design:**

- Solar plant power capacity [MW] (`solar_MW`)

**Battery Storage design:**

- Battery power [MW] (`b_P`)
- Battery energy capacity in hours [MWh] (`b_E_h `): Battery storage capacity in hours of full battery power (`b_E = b_E_h * b_P `). 
- Cost of battery power fluctuations in peak price ratio [-] (`cost_of_batt_degr`): This parameter controls how much penalty is given to do ramps in battery power in the HPP operation.


### Possible objective functions

- LCOE: Levelized cost of energy.
- IRR: Internal rate of return. It is not defined for projects that produce negative Net present values. Hydesigb returns IRR = 0 if the NPV < 0. Nevertheless, optimizations can be problematic for sites without a clear case. 
- NPV/CAPEX: Net present value over total CAPEX. A good proxy variable, that will produce optimal sites with the optimal IRR, but that is defined on sites with negative NPV.

The available variables for optimiaztion are:

```
 'NPV_over_CAPEX',
 'NPV [MEuro]',
 'IRR',
 'LCOE [Euro/MWh]',
 'CAPEX [MEuro]',
 'OPEX [MEuro]',
 'penalty lifetime [MEuro]',
 ```
 

In [1]:
# Install hydesign if needed
import importlib
if not importlib.util.find_spec("hydesign"):
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git    

In [2]:
import pandas as pd
from hydesign.EGO_surrogate_based_optimization import EGO_path
from hydesign.examples import examples_filepath

# Optimize using EGO

In [3]:
examples_sites = pd.read_csv(f'{examples_filepath}examples_sites.csv', index_col=0)
examples_sites

,case,name,longitude,latitude,altitude,input_ts_fn,sim_pars_fn,price_fn,price_col
0,India,Indian_site_good_wind,77.500226,8.334294,679.803454,India/input_ts_Indian_site_good_wind.csv,India/hpp_pars.yml,India/Indian_elec_price_t.csv,Price
1,India,Indian_site_good_solar,68.542204,23.542099,29.883557,India/input_ts_Indian_site_good_solar.csv,India/hpp_pars.yml,India/Indian_elec_price_t.csv,Price
2,India,Indian_site_bad_solar_bad_wind,77.916878,17.292316,627.424643,India/input_ts_Indian_site_bad_solar_bad_wind.csv,India/hpp_pars.yml,India/Indian_elec_price_t.csv,Price
3,Europe,France_good_solar,4.229736,44.422011,204.000000,Europe/input_ts_France_good_solar.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,FR_R
4,Europe,France_good_wind,-0.864258,48.744116,302.000000,Europe/input_ts_France_good_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,FR_R
5,Europe,France_bad_solar_n_wind,2.167969,47.428087,140.000000,Europe/input_ts_France_bad_solar_n_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,FR_R
6,Europe,Germany_bad_solar_n_wind,10.766602,49.310798,442.000000,Europe/input_ts_Germany_bad_solar_n_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DE_ME
7,Europe,Germany_good_wind,7.873535,53.287111,5.000000,Europe/input_ts_Germany_good_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DE_NW
8,Europe,Denmark_good_solar,11.813965,55.397760,42.000000,Europe/input_ts_Denmark_good_solar.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DK_E
9,Europe,Denmark_good_wind,8.594398,56.227322,85.000000,Europe/input_ts_Denmark_good_wind.csv,Europe/hpp_pars.yml,Europe/2030-EL_PRICE.csv,DK_W


In [4]:
EGO_simple = f'{EGO_path}EGO_surrogate_based_optimization_simple_hpp.py'

In [5]:
%run $EGO_simple --help

usage: EGO_surrogate_based_optimization_simple_hpp.py [-h] [--example EXAMPLE]
                                                      [--name NAME]
                                                      [--longitude LONGITUDE]
                                                      [--latitude LATITUDE]
                                                      [--altitude ALTITUDE]
                                                      [--input_ts_fn INPUT_TS_FN]
                                                      [--sim_pars_fn SIM_PARS_FN]
                                                      [--opt_var OPT_VAR]
                                                      [--rotor_diameter_m ROTOR_DIAMETER_M]
                                                      [--hub_height_m HUB_HEIGHT_M]
                                                      [--wt_rated_power_MW WT_RATED_POWER_MW]
                                                      [--surface_tilt_deg SURFACE_TILT_DEG]
                       

#### 

In [6]:
%run $EGO_simple \
    --example 10 \
    --opt_var "NPV_over_CAPEX"\
    --rotor_diameter_m 100\
    --hub_height_m 120\
    --wt_rated_power_MW 2\
    --surface_tilt_deg 20\
    --surface_azimuth_deg 180\
    --DC_AC_ratio 1\
    --num_batteries 1\
    --n_procs 31\
    --n_doe 31\
    --n_clusters 15\
    --n_seed 0\
    --max_iter 10\
    --final_design_fn 'hydesign_simple_design_10.csv'

Selected example site:
---------------------------------------------------
case                                    Europe
name                               Netherlands
longitude                                6.833
latitude                                 53.45
altitude                                   2.0
input_ts_fn    Europe/input_ts_Netherlands.csv
sim_pars_fn                Europe/hpp_pars.yml
price_fn              Europe/2030-EL_PRICE.csv
price_col                                 NL_R
Name: 10, dtype: object




Sizing a HPP plant at Netherlands:

longitude = 6.833
latitude = 53.45
altitude = 2.0

rotor_diameter_m = 100.0
hub_height_m = 120.0
wt_rated_power_MW = 2.0
surface_tilt_deg = 20.0
surface_azimuth_deg = 180.0
DC_AC_ratio = 1.0



Initial 31 simulations took 0.23 minutes



/home/gupta/miniconda3/envs/hydesign/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Update sm and extract candidate points took 0.06 minutes
Check-optimal candidates: new 17 simulations took 0.21 minutes
  rel_yopt_change = -2.07E-01
Iteration 1 took 0.28 minutes



/home/gupta/miniconda3/envs/hydesign/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Update sm and extract candidate points took 0.07 minutes
Check-optimal candidates: new 23 simulations took 0.21 minutes
  rel_yopt_change = -1.80E-02
Iteration 2 took 0.29 minutes



...

Update sm and extract candidate points took 0.11 minutes
Check-optimal candidates: new 23 simulations took 0.23 minutes
  rel_yopt_change = 0.00E+00
Iteration 8 took 0.38 minutes



/home/gupta/miniconda3/envs/hydesign/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Update sm and extract candidate points took 0.12 minutes
Check-optimal candidates: new 24 simulations took 0.22 minutes
  rel_yopt_change = 0.00E+00
Iteration 9 took 0.39 minutes



/home/gupta/miniconda3/envs/hydesign/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Update sm and extract candidate points took 0.13 minutes
Check-optimal candidates: new 25 simulations took 0.22 minutes
  rel_yopt_change = 0.00E+00
Iteration 10 took 0.39 minutes

Surrogate based optimization is converged.

Design:
---------------
Nwt: 91
wind_MW_per_km2 [MW/km2]: 5.048
solar_MW [MW]: 460
b_P [MW]: 6
b_E_h [h]: 2
cost_of_battery_P_fluct_in_peak_price_ratio: 0.290


NPV_over_CAPEX: 0.555
NPV [MEuro]: 217.563
IRR: 0.103
LCOE [Euro/MWh]: 28.686
CAPEX [MEuro]: 392.018
OPEX [MEuro]: 7.284
penalty lifetime [MEuro]: 0.000
AEP [GWh]: 1243.383
GUF: 0.473
grid [MW]: 300.000
wind [MW]: 182.000
solar [MW]: 460.000
Battery Energy [MWh]: 12.000
Battery Power [MW]: 6.000
Total curtailment [GWh]: 674.978
Awpp [km2]: 36.052
Rotor diam [m]: 100.000
Hub height [m]: 120.000
Number_of_batteries: 1.000

Optimization with 10 iterations and 251 model evaluations took 3.83 minutes



In [7]:
%run $EGO_simple \
    --example 9 \
    --opt_var "LCOE [Euro/MWh]"\
    --rotor_diameter_m 100\
    --hub_height_m 120\
    --wt_rated_power_MW 2\
    --surface_tilt_deg 20\
    --surface_azimuth_deg 180\
    --DC_AC_ratio 1\
    --num_batteries 2\
    --n_procs  31\
    --n_doe 31\
    --n_clusters 15\
    --n_seed 0\
    --max_iter 10\
    --final_design_fn 'hydesign_simple_design_9.csv'

Selected example site:
---------------------------------------------------
case                                          Europe
name                               Denmark_good_wind
longitude                                   8.594398
latitude                                   56.227322
altitude                                        85.0
input_ts_fn    Europe/input_ts_Denmark_good_wind.csv
sim_pars_fn                      Europe/hpp_pars.yml
price_fn                  Europe/2030-EL_PRICE_1.csv
price_col                                       DK_W
Name: 9, dtype: object




Sizing a HPP plant at Denmark_good_wind:

longitude = 8.594398
latitude = 56.227322
altitude = 85.0

rotor_diameter_m = 100.0
hub_height_m = 120.0
wt_rated_power_MW = 2.0
surface_tilt_deg = 20.0
surface_azimuth_deg = 180.0
DC_AC_ratio = 1.0



Initial 31 simulations took 0.23 minutes



/home/gupta/miniconda3/envs/hydesign/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Update sm and extract candidate points took 0.06 minutes
Check-optimal candidates: new 16 simulations took 0.22 minutes
  rel_yopt_change = 7.50E-02
Iteration 1 took 0.3 minutes



/home/gupta/miniconda3/envs/hydesign/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Update sm and extract candidate points took 0.07 minutes
Check-optimal candidates: new 20 simulations took 0.22 minutes
  rel_yopt_change = 4.15E-03
Iteration 2 took 0.3 minutes



...

Update sm and extract candidate points took 0.11 minutes
Check-optimal candidates: new 21 simulations took 0.22 minutes
  rel_yopt_change = 2.81E-05
Iteration 8 took 0.37 minutes



/home/gupta/miniconda3/envs/hydesign/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Update sm and extract candidate points took 0.12 minutes
Check-optimal candidates: new 23 simulations took 0.21 minutes
  rel_yopt_change = 0.00E+00
Iteration 9 took 0.38 minutes



/home/gupta/miniconda3/envs/hydesign/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Update sm and extract candidate points took 0.13 minutes
Check-optimal candidates: new 26 simulations took 0.23 minutes
  rel_yopt_change = 0.00E+00
Iteration 10 took 0.4 minutes


Design:
---------------
Nwt: 138
wind_MW_per_km2 [MW/km2]: 5.047
solar_MW [MW]: 180
b_P [MW]: 0
b_E_h [h]: 23
cost_of_battery_P_fluct_in_peak_price_ratio: 12.567


NPV_over_CAPEX: 0.297
NPV [MEuro]: 132.806
IRR: 0.081
LCOE [Euro/MWh]: 29.299
CAPEX [MEuro]: 447.160
OPEX [MEuro]: 8.539
penalty lifetime [MEuro]: 0.000
AEP [GWh]: 1402.462
GUF: 0.534
grid [MW]: 300.000
wind [MW]: 276.000
solar [MW]: 180.000
Battery Energy [MWh]: 0.000
Battery Power [MW]: 0.000
Total curtailment [GWh]: 262.551
Awpp [km2]: 54.687
Rotor diam [m]: 100.000
Hub height [m]: 120.000
Number_of_batteries: 2.000

Optimization with 10 iterations and 248 model evaluations took 3.73 minutes



# Size a HPP plant based on a detailed hpp model

This **static** notebook illustrates how to solve hybrid power plant sizing optimization in a specific location based on preselected turbine type.

**To execute this setup a server is required as it relies on paralle evaluation of the model.**

Sizing a hybrid power plant consists on designing the following parameters:

**Wind Turbine design:**

1. Clearance [m] (`clearance`): Height from the ground to rotor tip at lowest posstion. This parameter controls the wind turbine hub height given a rotor radius: `clearance =  HH - R`. 
2. Specific power of the wind turbine [MW/m2] (`sp`): Defined as `sp = p_rated / ( pi * R^2 )`, it controls the turbine rotor size for a given rated power. Turbines with lower specific power produce more power at lower wind speeds, but are more expensive.
3. Rated power of the wind turbine [MW] (`p_rated`)

**Wind Plant design:**

4. Number of wind turbines in the wind plant [-] (`Nwt`)
5. Wind power installation density [MW/km2] (`wind_MW_per_km2`): This parameter controls how closely spaced are the turbines, which in turns affect how much wake losses are present.

**PV Plant design:**

6. Solar plant power capacity [MW] (`solar_MW`)
7. Surface tilt [deg] (`surface_tilt`)
8. Surface azimuth [deg] (`surface_azimuth`)
9. DC-AC ratio [-] (`solar_DCAC`): This parameter controls how much over-planting of PV (in DC power) is connected to the inverters. It is common practice in PV design to have `solar_DCAC = 1.5`. 

**Battery Storage design:**

10. Battery power [MW] (`b_P`)
11. Battery energy capacity in hours [MWh] (`b_E_h `): Battery storage capacity in hours of full battery power (`b_E = b_E_h * b_P `). 
12. Cost of battery power fluctuations in peak price ratio [-] (`cost_of_batt_degr`): This parameter controls how much penalty is given to do ramps in battery power in the HPP operation.


### Possible objective functions

- LCOE: Levelized cost of energy.
- IRR: Internal rate of return. It is not defined for projects that produce negative Net present values. Hydesigb returns IRR = 0 if the NPV < 0. Nevertheless, optimizations can be problematic for sites without a clear case. 
- NPV/CAPEX: Net present value over total CAPEX. A good proxy variable, that will produce optimal sites with the optimal IRR, but that is defined on sites with negative NPV.

The available variables for optimiaztion are:

```
 'NPV_over_CAPEX',
 'NPV [MEuro]',
 'IRR',
 'LCOE [Euro/MWh]',
 'CAPEX [MEuro]',
 'OPEX [MEuro]',
 'penalty lifetime [MEuro]',
 ```
 

In [5]:
EGO_full = f'{EGO_path}EGO_surrogate_based_optimization.py'

In [6]:
%run $EGO_full --help

usage: EGO_surrogate_based_optimization.py [-h] [--example EXAMPLE]
                                           [--name NAME]
                                           [--longitude LONGITUDE]
                                           [--latitude LATITUDE]
                                           [--altitude ALTITUDE]
                                           [--input_ts_fn INPUT_TS_FN]
                                           [--sim_pars_fn SIM_PARS_FN]
                                           [--opt_var OPT_VAR]
                                           [--num_batteries NUM_BATTERIES]
                                           [--n_procs N_PROCS] [--n_doe N_DOE]
                                           [--n_clusters N_CLUSTERS]
                                           [--n_seed N_SEED]
                                           [--max_iter MAX_ITER]
                                           [--final_design_fn FINAL_DESIGN_FN]

options:
  -h, --help            show this h

In [7]:
%run $EGO_full \
    --example 9 \
    --opt_var "NPV_over_CAPEX"\
    --num_batteries 1\
    --n_procs  1\
    --n_doe 31\
    --n_clusters 2\
    --n_seed 0\
    --max_iter 10\
    --final_design_fn 'hydesign_design_9.csv'

Selected example site:
---------------------------------------------------
case                                          Europe
name                               Denmark_good_wind
longitude                                   8.594398
latitude                                   56.227322
altitude                                        85.0
input_ts_fn    Europe/input_ts_Denmark_good_wind.csv
sim_pars_fn                      Europe/hpp_pars.yml
price_fn                    Europe/2030-EL_PRICE.csv
price_col                                       DK_W
Name: 9, dtype: object




Sizing a HPP plant at Denmark_good_wind:

longitude = 8.594398
latitude = 56.227322
altitude = 85.0



Initial 31 simulations took 6.29 minutes



/home/gupta/miniconda3/envs/hydesign/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Update sm and extract candidate points took 0.02 minutes
Check-optimal candidates: new 4 simulations took 0.81 minutes
  rel_yopt_change = 0.00E+00
Iteration 1 took 0.84 minutes

Update sm and extract candidate points took 0.02 minutes


/home/gupta/miniconda3/envs/hydesign/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Check-optimal candidates: new 4 simulations took 0.54 minutes
  rel_yopt_change = 0.00E+00
Iteration 2 took 0.57 minutes



/home/gupta/miniconda3/envs/hydesign/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Update sm and extract candidate points took 0.02 minutes
Check-optimal candidates: new 3 simulations took 0.46 minutes
  rel_yopt_change = 0.00E+00
Iteration 3 took 0.5 minutes

Surrogate based optimization is converged.

Design:
---------------
clearance [m]: 13.000
sp [m2/W]: 265.000
p_rated [MW]: 3.000
Nwt: 99.000
wind_MW_per_km2 [MW/km2]: 6.975
solar_MW [MW]: 106.000
surface_tilt [deg]: 10.500
surface_azimuth [deg]: 153.833
DC_AC_ratio: 1.069
b_P [MW]: 35.000
b_E_h [h]: 8.000
cost_of_battery_P_fluct_in_peak_price_ratio: 16.432


NPV_over_CAPEX: 0.572
NPV [MEuro]: 211.969
IRR: 0.107
LCOE [Euro/MWh]: 24.762
CAPEX [MEuro]: 370.359
OPEX [MEuro]: 7.207
penalty lifetime [MEuro]: 0.000
AEP [GWh]: 1395.010
GUF: 0.531
grid [MW]: 300.000
wind [MW]: 297.000
solar [MW]: 106.000
Battery Energy [MWh]: 280.000
Battery Power [MW]: 35.000
Total curtailment [GWh]: 89.505
Awpp [km2]: 42.583
Rotor diam [m]: 120.058
Hub height [m]: 73.029
Number_of_batteries: 1.000

Optimization with 3 iterations and 4